In [302]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

In [303]:
df=pd.read_csv("./TitanicDataset.csv")
df=df[["Pclass","Age","SibSp","Parch","Fare","Survived"]]
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Fare'].fillna(df['Fare'].median(), inplace=True)
df[:6]

,Pclass,Age,SibSp,Parch,Fare,Survived
0,3,22.0,1,0,7.2500,0
1,1,38.0,1,0,71.2833,1
2,3,26.0,0,0,7.9250,1
3,1,35.0,1,0,53.1000,1
4,3,35.0,0,0,8.0500,0
5,3,28.0,0,0,8.4583,0


In [304]:
def calculate_prior(df, Y):
    classes = sorted(list(df[Y].unique()))
    prior = []
    for i in classes:
        prior.append(len(df[df[Y]==i])/len(df))
    return prior

In [305]:
def calculate_likelihood_gaussian(df, feat_name, feat_val, Y, label):
  feat = list(df.columns)[:-1]
  df = df[df[Y]==label]
  mean, std = df[feat_name].mean(), df[feat_name].std()
  p_x_given_y = (1 / (np.sqrt(2 * np.pi) * std)) * np.exp(-
  ((feat_val-mean)**2 / (2 * std**2 )))
  return p_x_given_y


In [306]:
def naive_bayes_gaussian(df, X, Y):
# get feature names
  features = list(df.columns)[:-1]

  # calculate prior
  prior = calculate_prior(df, Y)

  Y_pred = []
  # loop over every data sample
  for x in X:
  # calculate likelihood
    labels = sorted(list(df[Y].unique()))
    likelihood = [1]*len(labels)
    for i in range(len(labels)):
      for j in range(len(features)):
        likelihood[i] *= calculate_likelihood_gaussian(df, features[j], x[j], Y, labels[i])

    # calculate posterior probability (numerator only)
    post_prob = [1]*len(labels)
    for i in range(len(labels)):
      post_prob[i] = likelihood[i] * prior[i]
    Y_pred.append(np.argmax(post_prob))
  print(likelihood)
 

  return np.array(Y_pred)

In [307]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(df,test_size=0.3)
X_test=test.iloc[:,:-1].values
Y_test=test.iloc[:,-1].values
Y_pred=naive_bayes_gaussian(train,X_test,Y="Survived")
print(Y_pred)


[1.8106960977360323e-09, 4.905024154700961e-07]
[1 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 1 1 0 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 1 1 0 0 0 1 0 0 1 0 0 0 0
 1 1 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1
 1 1 0 1 0 1 0 0 0 0 0 1 1 0 1 1 1 1 0 1 1 1 1 0 1 0 0 0 1 0 0 0 0 1 0 1 1
 1 0 0 0 0 1 1 0 1]


In [308]:
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_pred,Y_test)*100)

72.38805970149254
